In [1]:
import duckdb
import geopandas as gpd
import shapely
from shapely.errors import WKBReadingError

# Connect to DuckDB
con = duckdb.connect()
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")


C:\Users\edoar\AppData\Local\Temp\ipykernel_29668\2950509256.py:4: FutureWarning: WKBReadingError is deprecated and will be removed in a future version. Use ShapelyError instead (functions previously raising {name} will now raise a ShapelyError instead).
  from shapely.errors import WKBReadingError


In [2]:

# Define bbox
minx, miny, maxx, maxy = 2.321001, 48.837379, 2.373657, 48.870248

In [3]:
df = con.execute(f"""
    SELECT *, ST_AsText(geometry) AS wkt_geom
    FROM read_parquet('https://data.source.coop/vida/google-microsoft-osm-open-buildings/geoparquet/by_country/country_iso=FRA/FRA.parquet')
    WHERE ST_Within(
        geometry,
        ST_MakeEnvelope({minx}, {miny}, {maxx}, {maxy})
    )
""").df()

import shapely.wkt
df["geometry"] = df["wkt_geom"].apply(shapely.wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
# Save as GeoJSON
gdf.to_file("fra_buildings4.geojson", driver="GeoJSON")

# Load again if needed
gdf2 = gpd.read_file("fra_buildings.geojson")